## Step 10-Getting Individual Job Descriptions ##

You will recall that we have aggregated job descriptions from several companies in order for us to get a 
regional data set. 

In this phase of the project we will see if we can now split up this aggregated data just to get individual
job descriptions for each company. We want to do this so that we can compare each individual job against the resume versions. 

We will be going through the same process of scraping and getting slugs. 

However, where we deviate in this process is that we will group all the "Paragraphs", "Bullets", "Bolded Items" and "Summary Text" against each respective company that produced them. 

We will use asterisks to separate each company job description from each other. 

In [14]:
import requests
import bs4
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()

In [15]:
pd.read_csv('Capstone_v1')

,Unnamed: 0,location,title,company,salary,summary,age,slugs
0,0,"Johns Creek, GA 30097","Data Scientist (PYTHON, HADOOP)",SoftVision - TAMS,NaN,3-5 years of experience building l...,4 days ago,/rc/clk?jk=1251fe14f961460a&fccid=8e4ade64e8fe...
1,1,"Atlanta, GA",Data Scientist,Cardinal Solutions,NaN,"What Data Scientists Do. Analyze, ...",1 day ago,/rc/clk?jk=0893fa69b25d5c08&fccid=1d9db135c2c7...
2,2,"Atlanta, GA",Data Scientist,Alamedatech Solutions LLC.,NaN,Experience in programming language...,2 days ago,/company/Alamedatech-Solutions-LLC./jobs/Data-...
3,3,"Atlanta, GA 30303 (Five Points area)",Data Analyst,Operation HOPE,NaN,Able to understand various data st...,30+ days ago,/company/Operation-Hope/jobs/Data-Analyst-6866...
4,4,"Atlanta, GA",Data Scientist,Equifax,NaN,Developing data driven models to q...,23 days ago,/rc/clk?jk=04f8b1a515e0db62&fccid=34c446722f3c...
5,5,"Atlanta, GA 30309 (Midtown area)",Fall 2018 Data Scientist/Machine Learning Inte...,Norfolk Southern Corp,NaN,You will have the opportunity of w...,30+ days ago,/rc/clk?jk=105f1bdc8b3513d3&fccid=2fc6cbacc711...
6,6,"Atlanta, GA",Data Scientist,SalesLoft,NaN,We are looking for a Data Scientis...,16 days ago,/rc/clk?jk=d1e588de6f01536d&fccid=de4fd2d39ebd...
7,7,"Atlanta, GA 30301 (Buckhead area)",Data Scientist,The Coca-Cola Company,NaN,Excellent data wrangling skills; S...,30+ days ago,/rc/clk?jk=1761018962b52cb3&fccid=681e47a680e0...
8,8,"Atlanta, GA",Data Scientist,Catalina Marketing,NaN,"We use advanced machine learning, ...",8 days ago,/rc/clk?jk=ba1b24901c04182c&fccid=7cb8926c7903...
9,9,"Atlanta, GA",Flight Safety Data Analysis Fall Co-Op,Delta Air Lines,NaN,The successful candidate will be r...,27 days ago,/rc/clk?jk=8fe4025946bb4187&fccid=af0caae05772...


In [28]:
df = pd.read_csv('Capstone_v1')

In [29]:
df.sort_values(by=['age'])

,Unnamed: 0,location,title,company,salary,summary,age,slugs
1,1,"Atlanta, GA",Data Scientist,Cardinal Solutions,NaN,"What Data Scientists Do. Analyze, ...",1 day ago,/rc/clk?jk=0893fa69b25d5c08&fccid=1d9db135c2c7...
35,35,"Atlanta, GA 30342 (Buckhead area)",Quantitative Analyst,GreenSky,NaN,"Exposure to SAS, SQL, Python, R/Ca...",10 days ago,/rc/clk?jk=41a3604e14e625e7&fccid=03416f3f2828...
11,11,"Atlanta, GA",Data Scientist,Inspire Brands,NaN,Data cleansing/processing -- work with IT to c...,10 days ago,/rc/clk?jk=08cf1b7ada36679c&fccid=1b082641b237...
46,46,"Alpharetta, GA","Manager, Data Scientist - Strategic Pricing (N...",CHEP,NaN,The scientist will be working with...,13 days ago,/rc/clk?jk=c9665029ef80e85b&fccid=9b228314b48d...
31,31,"Atlanta, GA","VP, Data Analytics",TCGplayer,NaN,VP of Data Analytics. Proven track...,15 days ago,/rc/clk?jk=0f0c98552e115529&fccid=27b200d3b82e...
6,6,"Atlanta, GA",Data Scientist,SalesLoft,NaN,We are looking for a Data Scientis...,16 days ago,/rc/clk?jk=d1e588de6f01536d&fccid=de4fd2d39ebd...
47,47,"Atlanta, GA",Director of Data Science,Prestige Staffing,NaN,"Leadership, Data Science, Quantita...",17 days ago,/company/Prestige-Staffing/jobs/Director-Data-...
13,13,"Atlanta, GA 30308 (Old Fourth Ward area)",Data Scientist III,Honeywell,NaN,"Data mining (including data auditing, aggregat...",17 days ago,/rc/clk?jk=d483893a183d6550&fccid=50208b5bf45e...
40,40,"Atlanta, GA",Chief Deep Learning Scientist,YIWEI Medical Inc.,NaN,Combine medical imaging models int...,18 days ago,"/company/YIWEI-Medical-Technology-Co.,-Ltd/job..."
2,2,"Atlanta, GA",Data Scientist,Alamedatech Solutions LLC.,NaN,Experience in programming language...,2 days ago,/company/Alamedatech-Solutions-LLC./jobs/Data-...


In [18]:
slugs = df['slugs']

In [19]:
extended_descriptions = []

for r in slugs[:10]:
    new_url = 'http://www.indeed.com' + r
    print('Requesting content from ' + new_url) # you can add + '...' 
    res = requests.get(new_url)
    # print('Converting content from the res object.')
    soup = BeautifulSoup(res.content, 'lxml')
    extended_descriptions.append(soup)
    sleep(3)
    print('Appending soup...')

Requesting content from http://www.indeed.com/rc/clk?jk=1251fe14f961460a&fccid=8e4ade64e8feb2ea&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/rc/clk?jk=0893fa69b25d5c08&fccid=1d9db135c2c77841&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/company/Alamedatech-Solutions-LLC./jobs/Data-Scientist-c9b0afc24b95d2db?fccid=a4ddacf5eec0d9cf&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/company/Operation-Hope/jobs/Data-Analyst-6866fbc693d5279b?fccid=63e09b63b5df262b&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/rc/clk?jk=04f8b1a515e0db62&fccid=34c446722f3cf576&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/rc/clk?jk=105f1bdc8b3513d3&fccid=2fc6cbacc7119b63&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/rc/clk?jk=d1e588de6f01536d&fccid=de4fd2d39ebd948a&vjs=3
Appending soup...
Requesting content from http://www.indeed.com/rc/clk?jk=1761018962b52cb3&fccid=681e4

In [20]:
extended_descriptions

[<!DOCTYPE html>
 <html dir="ltr" lang="en">
 <head><title>Data Scientist (PYTHON, HADOOP) job - SoftVision - TAMS - Johns Creek, GA | Indeed.com</title>
 <meta content="July 23, 2018 - Data Scientist (PYTHON, HADOOP) - WE ARE A TRANSFORMATIONAL PARTNER  br We marry design and engineering language in ways that produce impactful and memorable experience journeys. We" name="description"/>
 <meta content="Data Scientist (PYTHON, HADOOP) job - SoftVision - TAMS - Johns Creek, GA" name="keywords"/>
 <meta content="NOINDEX" name="robots"/>
 <link href="/m/viewjob?jk=1251fe14f961460a" media="only screen and (max-width: 640px)" rel="alternate"/>
 <link href="/m/viewjob?jk=1251fe14f961460a" media="handheld" rel="alternate"/>
 <meta content="origin-when-cross-origin" name="referrer"/>
 <meta content="115882278440564" property="fb:app_id"/>
 <meta content="Data Scientist (PYTHON, HADOOP) job - SoftVision - TAMS - Johns Creek, GA" property="og:title"/>
 <meta content="Indeed.com" property="og:site

In [21]:
len(extended_descriptions)

10

In [22]:
for each in extended_descriptions:
    print(type(each))

<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>
<class 'bs4.BeautifulSoup'>


In [24]:
for each in extended_descriptions:
    print(each.find('span', attrs={'class': 'company'}).text)

SoftVision - TAMS
Cardinal Solutions
Alamedatech Solutions LLC.
Operation HOPE


AttributeError: 'NoneType' object has no attribute 'text'

In [25]:
for each in extended_descriptions:
    
    print(each.find('span', attrs={'summary'}).text)

WE ARE A TRANSFORMATIONAL PARTNER
We marry design and engineering language in ways that produce impactful and memorable experience journeys. We partner all the way to continuously improve our clients’ digital maturity. Our Studio network brings the optimal combination of skill, scale, and cost for each stage of the product development lifecycle. And to do this we need great transformational people that want to impact the projects and organizations that they work with.

We are looking for an exceptional Data Scientist (PYTHON, HADOOP) to work with our cross-functional team onsite at our client in the North Atlanta area. Core to this need are:


5-7 years of Data Modeling and Machine Learning Experience
5-7 years of Python & R- Language Experience
5-7 years of HADOOP-BIGDATA experience
Proficiency dealing with extraction and manipulation of relational databases (SQL)
3 -5 years of experience in advanced statistical techniques such as predictive statistical models, customer profiling, seg

AttributeError: 'NoneType' object has no attribute 'text'

In [26]:
all_jobs = {}

for ix, page in enumerate(extended_descriptions):
    
    print('\n\n************************\n\n')
    
    all_jobs[ix] = {
        'company':[],
        'bold_items':[],
        'paragraphs':[],
        'bullets': [],
        'addtl_comments': [],
        
        
    }
    
    for each in page.find_all('title'):
        all_jobs[ix]['company'].append(each.get_text())
        # company.append(each.get_text())
        # print(each.get_text())
        
    for each in page.find_all('b'):
        all_jobs[ix]['bold_items'].append(each.get_text())
        #single_job.append(each.get_text())
        print(each.get_text())

    for each in page.find_all('p'):
        all_jobs[ix]['paragraphs'].append(each.get_text())
        print(each.get_text())
        
    for each in page.find_all('li'):
        all_jobs[ix]['bullets'].append(each.get_text())
        print(each.get_text())
        
    for each in page.find_all('span', attrs={'summary'}):
        all_jobs[ix]['addtl_comments'].append(each.get_text())
        print(each.get_text())   
        
        
  
        



************************


Data Scientist (PYTHON, HADOOP)
5-7 years of Data Modeling and Machine Learning Experience
5-7 years of Python & R- Language Experience
5-7 years of HADOOP-BIGDATA experience
Proficiency dealing with extraction and manipulation of relational databases (SQL)
3 -5 years of experience in advanced statistical techniques such as predictive statistical models, customer profiling, segmentation analysis survey design and analysis and data mining
3-5 years of experience of learning systems to analyze and filter continuous data flows.
3-5 years of experience of analyzing large data sets to develop custom models and algorithms to drive business solutions.
3-5 years of experience building large data sets from multiple sources to build algorithms for predicting future data characteristics.
Experience developing custom data models to drive recommendations.
Experience working in a cross-functional environment.
Excellent Communication Skills
Good Team Player
Able to work i

In [ ]:
all_jobs
for each in all_jobs:
    print(all_jobs[each]['company'])
    print(all_jobs[each]['paragraphs'])
    print(all_jobs[each]['bold_items'])
    print(all_jobs[each]['bullets'])
    print(all_jobs[each]['addtl_comments'])
    
    print('\n\n*****\n\n')